In [1]:
import pandas as pd
import plotly.express as px
import dash
from dash import Dash, dcc, html, Input, Output,callback

In [2]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
lunch_site=list(df['Launch Site'].unique())
lunch_site.append("All")
lunch_site

['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40', 'All']

In [4]:
Payload_max=df['Payload Mass (kg)'].max()
Payload_min=df['Payload Mass (kg)'].min()

In [5]:
app=dash.Dash(__name__)

app.layout=html.Div(children=[html.H1('SpaceX Launch Records Dashboard',style={'testAlighn':'cetnter'})
                    ,html.Div(['Select Launch site:',dcc.Dropdown(lunch_site,'All',id='Lunch_site')])
                    ,html.Div([html.Div(dcc.Graph(id='Success_plot'))])                
                    ,html.Div(['Payload mass:',dcc.RangeSlider(min=Payload_min,max=Payload_max,value=[Payload_min,Payload_max],id='payload')])
                    ,html.Div([html.Div(dcc.Graph(id='payload-plot'))])
                                  ])

In [6]:
@app.callback([Output(component_id='Success_plot', component_property='figure')],
              Input(component_id='Lunch_site', component_property='value')
             )

def my_pie(site_name):
    if site_name=='All':
        pie_fig=px.pie(data_frame=df,names='class')
    else:
        pie_fig=px.pie(data_frame=df[df['Launch Site']==site_name],names='class')
    return [pie_fig]

    


In [7]:
@app.callback([Output(component_id='payload-plot', component_property='figure')],
              [Input(component_id='Lunch_site', component_property='value'),
               Input(component_id='payload', component_property='value')]
             )

def my_payload(site_name,load):
    min_load,max_load=load
    if site_name=='All':
        payload_fig=px.scatter(df[(df['Payload Mass (kg)']>=min_load) & (df['Payload Mass (kg)']<=max_load)],x='Payload Mass (kg)',y='class',color='Booster Version')
        
    else:
        payload_fig=px.scatter(df[(df['Launch Site']==site_name) & (df['Payload Mass (kg)']>=min_load) & (df['Payload Mass (kg)']<=max_load)],x='Payload Mass (kg)',y='class',color='Booster Version')
    payload_fig.update_layout(yaxis=dict(tickvals=[0, 1]))
    return [payload_fig]

In [8]:
if __name__=='__main__':
    app.run_server(debug=True)